In [3]:
import random

from unsloth import FastLanguageModel

ImportError: Pytorch is not installed. Go to https://pytorch.org/.
We have some installation instructions on our Github page.

In [2]:
import inspect


print(
    inspect.getsource(
        FastLanguageModel
    )
)

NameError: name 'FastLanguageModel' is not defined

In [6]:
import os
from groq import Groq
from dotenv import load_dotenv, find_dotenv

load_dotenv(
    
    find_dotenv()
)

client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )

# test the LLM

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models are artificial intelligence (AI) models that are designed to quickly process and generate human-like language, often for real-time applications. Their importance lies in several areas:

1. **Conversational AI**: Fast language models power many conversational AI systems, such as virtual assistants, chatbots, and customer service agents. They enable these systems to respond rapidly to user queries, making interactions feel more natural and efficient.
2. **Real-time processing**: Fast language models can process and generate text rapidly, making them suitable for applications that require instantaneous responses, such as:
	* Live chat support
	* Real-time translation
	* Sentiment analysis
3. **Inferential generation**: Fast language models can generate text quickly, allowing them to infer missing information or complete sentences, which is particularly useful in:
	* Question answering
	* Text summarization
	* Opinion extraction
4. **Large-scale text analysis**: Fast l

# QA synthetic generation

In [8]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

# get the docs

In [12]:
from datasets import load_dataset

ds = load_dataset("m-ric/huggingface_doc", split="train")

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [18]:
ds.take(1)

Dataset({
    features: ['text', 'source'],
    num_rows: 1
})

In [16]:
from langchain.docstore.document import Document
import inspect


print(
    
    inspect.getsource(
        Document
    )
)


class Document(Serializable):
    """Class for storing a piece of text and associated metadata."""

    page_content: str
    """String text."""
    metadata: dict = Field(default_factory=dict)
    """Arbitrary metadata about the page content (e.g., source, relationships to other
        documents, etc.).
    """
    type: Literal["Document"] = "Document"

    def __init__(self, page_content: str, **kwargs: Any) -> None:
        """Pass page_content in as positional or named arg."""
        super().__init__(page_content=page_content, **kwargs)

    @classmethod
    def is_lc_serializable(cls) -> bool:
        """Return whether this class is serializable."""
        return True

    @classmethod
    def get_lc_namespace(cls) -> List[str]:
        """Get the namespace of the langchain object."""
        return ["langchain", "schema", "document"]


In [20]:
_source = ds.take(1)['source']
_text = ds.take(1)['text']



In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm


langchain_docs = [LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(ds)]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

100%|██████████| 2647/2647 [00:00<00:00, 11119.95it/s]


In [23]:
len(docs_processed)

13841

In [30]:
docs_processed[0].__dict__.keys()

dict_keys(['page_content', 'metadata', 'type'])

In [40]:
import random

_pairs = []
for k in tqdm(
    random.sample(
        docs_processed,
        10
    )
):
    page_content = k.page_content
    metadata = k.metadata
    _prompt = QA_generation_prompt.format(context=page_content)
    qa_pair = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": _prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    
    _gen_text = qa_pair.choices[0].message.content
    
    quesion = _gen_text.split("Factoid question:")[1].split("Answer:")[0].strip()
    answer = _gen_text.split("Answer:")[1].strip()
    _pairs.append(
        {
            "question": quesion,
            "answer": answer,
            "context": page_content,
            "metadata": metadata['source']
        }
    )


100%|██████████| 10/10 [00:07<00:00,  1.40it/s]


In [38]:
metadata

{'source': 'huggingface/huggingface_hub/blob/main/docs/source/en/guides/manage-cache.md',
 'start_index': 0}

In [37]:
qa_pair.dict()

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Here is the factoid question and answer:\n\nFactoid question: What are the types of folders that all caching directories contain?\nAnswer: refs, blobs, snapshots',
    'role': 'assistant',
    'tool_calls': None}}],
 'id': 'chatcmpl-5d6cc272-7ec1-466d-af68-8647ec0ddf37',
 'created': 1715631463,
 'model': 'llama3-8b-8192',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_af05557ca2',
 'usage': {'completion_time': 0.039,
  'completion_tokens': 32,
  'prompt_time': 0.141,
  'prompt_tokens': 571,
  'queue_time': None,
  'total_time': 0.18,
  'total_tokens': 603},
 'x_groq': {'id': 'req_01hxssn6cef4297j1h802bt7wg'}}

# write the generated QA to yaml


In [45]:
import yaml
os.makedirs("data", exist_ok=True)
with open("data/qa_pairs.yaml", "w") as f:
    yaml.dump(_pairs, f)

In [46]:
import pandas as pd


pd.DataFrame.from_dict(_pairs)

,question,answer,context,metadata
0,What are some ideas of areas where I can contr...,Share exciting models with the community throu...,* **Where and how can I contribute?**\n \nIt ...,huggingface/blog/blob/main/fellowship.md
1,What is the top 1 accuracy of the model on the...,80.25%,Weights: https://github.com/rwightman/pytorch-...,huggingface/pytorch-image-models/blob/main/hfd...
2,What is the task set when initializing the `In...,feature-extraction,Some tasks might also require additional param...,huggingface/huggingface_hub/blob/main/src/hugg...
3,What is the model size of SetFit Roberta?,355 million,And just by switching out the base Sentence Tr...,huggingface/blog/blob/main/setfit.md
4,What is the theme used in the Gradio demo?,Base,Gradio Demo: diff_texts\n\n\n```\n!pip install...,gradio-app/gradio/blob/main/demo/diff_texts/ru...
5,Who fixed the issue mentioned in pull request ...,abidlabs,- [#6680](https://github.com/gradio-app/gradio...,gradio-app/gradio/blob/main/CHANGELOG.md
6,What is an available pipeline for multimodal t...,ImageToTextPipeline,[[autodoc]] VideoClassificationPipeline\n -...,huggingface/transformers/blob/main/docs/source...
7,How can you upload an Argilla dataset programm...,Using the argilla Python library.,"<div class=""flex justify-center"">\n<img src=""h...",huggingface/hub-docs/blob/main/docs/hub/spaces...
8,"What has rinna Co., Ltd. released models for?",Japanese text.,"## What’s Next?\nCompared to Stable Diffusion,...",huggingface/blog/blob/main/japanese-stable-dif...
9,What layers should be added to `modules_to_sav...,Newly initialized layers mentioned in a warnin...,"In PEFT, we try to correctly guess the `module...",huggingface/peft/blob/main/docs/source/develop...
